# Registro Sismico en Bolivia entre 1920 - 2024

<center>
    <img src="https://gitlab.com/ramisohj/data-science/-/raw/main/data/maps/bolivian-map.jpg?inline=false" alt="drawing" width="400"/>
</center>

## Description
Coleccion de datos acerca de los terremotos registrados entre los años 1920 y 2024.

## Source
* https://www.usgs.gov/
* https://earthquake.usgs.gov/earthquakes/search/

| Column | Data Type | Typical Values | Variable | Description |
| ----- | ----- | ----- | ----- | ----- |
| time | Date |  1970-01-01T00:00:00.000Z | CUANTITATIVA | Hora en la que se produjo el evento. |
| latitude | Decimal | [-90.0, 90.0] | CUANTITATIVA | Grados decimales de latitud. Valores negativos para latitudes meridionales. |
| longitude | Decimal | [-180.0, 180.0] | CUANTITATIVA | Grados decimales de longitud. Valores negativos para longitudes occidentales. |
| depth | Decimal | [0, 1000] | CUANTITATIVA | Profundidad del evento en kilómetros. |
| mag | Decimal | [-1.0, 10.0] | CUANTITATIVA | La magnitud del evento. |
| magType | String | [Md, Ml, Ms, Mw, Me, Mi, Mb, MLg] | CUALITATIVA | El método o algoritmo utilizado para calcular la magnitud preferida para el evento. |
| nst | Integer | | CUANTITATIVA | El número total de estaciones sísmicas utilizadas para determinar la ubicación del terremoto. |
| gap | Decimal | [0.0, 180.0] | CUANTITATIVA | La mayor brecha azimutal entre estaciones azimutalmente adyacentes (en grados). |
| dmin | Decimal | [0.4, 7.1] | CUANTITATIVA | Distancia horizontal desde el epicentro hasta la estación más cercana (en grados). 1 grado equivale aproximadamente a 111,2 kilómetros. |
| rms | Decimal | [0.13,1.39] | CUANTITATIVA | El tiempo de viaje residual de la media cuadrática (RMS), en segundos, utilizando todos los pesos. Este parámetro proporciona una medida del ajuste de los tiempos de llegada observados con los tiempos de llegada previstos para esta ubicación. |
| net | String | [ak, at, ci, hv, ld, mb, nc, nm, nn, pr, pt, se, us, uu, uw] | CUALITATIVA | Identificador del colaborador de datos. Identifica la red considerada como la fuente preferida de información para este evento. |
| id | String | usp00000aa | CUALITATIVA | Un identificador único para el evento. Este es el identificador preferido actual para el evento y puede cambiar con el tiempo. |
| updated | Date | 1970-01-01T00:00:00.000Z | CUANTITATIVA | Hora en que se actualizó el evento por última vez. |
| place | String | | CUALITATIVA | Descripción textual de la región geográfica mencionada cerca del evento. |
| type | String | [earthquake, quarry] | CUALITATIVA | Tipo de evento sísmico. |
| horizontalError | Decimal | [0, 100] | CUANTITATIVA | Incertidumbre de la ubicación reportada del evento en kilómetros. |
| depthError | Decimal | [0, 100] | CUANTITATIVA | Incertidumbre de la profundidad reportada del evento en kilómetros. |
| magError | Decimal | [0, 100] | CUANTITATIVA | Incertidumbre de la magnitud informada del evento. Error estándar estimado de la magnitud. |
| magNst | Integer | | CUANTITATIVA | El número total de estaciones sísmicas utilizadas para calcular la magnitud de este terremoto. |
| status | String | [automatic, reviewed, deleted] | CUALITATIVA | Indica si el evento ha sido revisado por un humano. |
| locationSource | String | [ak, at, ci, hv, ld, mb, nc, nm, nn, pr, pt, se, us, uu, uw] | CUALITATIVA | La red que originalmente fue autora de la ubicación reportada de este evento. |
| magSource | String | [ak, at, ci, hv, ld, mb, nc, nm, nn, pr, pt, se, us, uu, uw] | CUALITATIVA | Red que originalmente fue autora de la magnitud reportada para este evento. |ada para este evento. ||
|

## Eleccion de variable cualitativa
* mag (magnitud del terremoto en escala richter represantado en un numero decimal)

## 1. Analisis de intervalo -de confianza (0.90 - 0.95 - 0.99) de los ultimos 100 terremotos en Bolivia segun su magnitud

In [379]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm


########################################################################################################
# readind the CSV file
url = 'https://gitlab.com/ramisohj/data-science/-/raw/main/data/earthquake/earthquakes_bolivia_1920-2024.csv?inline=false'
data_earthquake = pd.read_csv(url)
########################################################################################################
# getting data from object
data_earthquake['time'] = (pd.to_datetime(data['time']))
########################################################################################################
# getting the last 100 records
data_100 = data_earthquake.tail(100)

# getting magnitude column
mag_data = data_100['mag']

# confidence intervals
trust_levels = [.9, .95, .99]

mag_mean = np.mean(mag_data)
mag_std = np.std(mag_data, ddof=1)
mag_size = len(mag_data)

print('mean: ', mag_mean)
print('std: ', mag_std)
print('size: ', mag_size)


stats.norm
# getting critical values
critical_values =  [norm.ppf((1 + confidence) / 2) for confidence in trust_levels]
# getting standard error
standard_error = mag_std / np.sqrt(mag_size)
# getting confidence intervals
confidence_intervals = [
    (mag_mean - critical_value * standard_error, mag_mean + critical_value * standard_error)
    for critical_value in critical_values
]
# print confidence intervals
for i, confidence_interval in enumerate(confidence_intervals):
    confidence_level = trust_levels[i]
    print(f'Confidence interval at {confidence_level*100}%: {confidence_interval}')



mean:  4.435
std:  0.2868762447432339
size:  100
Confidence interval at 90.0%: (4.387813056834787, 4.482186943165212)
Confidence interval at 95.0%: (4.378773289228316, 4.4912267107716835)
Confidence interval at 99.0%: (4.361105576229831, 4.508894423770168)


## 2. Test de Hipotesis de Sismos en Uyuni y Atocha

In [373]:
from scipy import stats

########################################################################################################
# check if earthquakes in Uyuni and Atocha are the same (100 last records only)
data_uyuni = data_earthquake[data_earthquake['place'].str.contains('Uyuni')].tail(200)
data_atocha = data_earthquake[data_earthquake['place'].str.contains('Atocha')].tail(200)

mag_uyuni = data_uyuni['mag']
mag_uyuni = mag_uyuni.reset_index(drop=True)
mag_atocha = data_atocha['mag']
mag_atocha = mag_atocha.reset_index(drop=True)


t_statistic = stats.ttest_1samp(mag_uyuni - mag_atocha, 0)
t_statistic_975 =  stats.t.ppf(0.975, mag_uyuni.shape[0]*2-2)


print("t-test:")
print("T-statistic:", t_statistic)
print('t_statistic_975:', t_statistic_975)
print('media= ',np.mean(mag_uyuni - mag_atocha))
print('mediana= ',np.median(mag_uyuni - mag_atocha))

#Confidence interval at 95%
print(t_statistic.confidence_interval(0.95))


t-test:
T-statistic: TtestResult(statistic=-1.0596085023495996, pvalue=0.29060715231175105, df=199)
t_statistic_975: 1.9659423239761926
media=  -0.028999999999999956
mediana=  0.0
ConfidenceInterval(low=-0.08296968753688289, high=0.024969687536882973)


### CONCLUSIONES:
#### Como podemos ver el p-valor es mayor a 0.05, debido a esto, no se rechazara la hipotesis nula, esto se debe a que ambas localidades se encuentran cerca de la cordillera de los Andes por tal motivo tienen similar comportamiento sismico.

## 3. Analisis ANOVA

In [362]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

from scipy.stats import levene

# check homogeneity
# group by magnitude type
group_m = df_mag_magType['mag'][df_mag_magType['magType']=='m']
group_mb = df_mag_magType['mag'][df_mag_magType['magType']=='mb']
group_md = df_mag_magType['mag'][df_mag_magType['magType']=='md']
group_ml = df_mag_magType['mag'][df_mag_magType['magType']=='ml']
group_ms = df_mag_magType['mag'][df_mag_magType['magType']=='ms']
group_mw = df_mag_magType['mag'][df_mag_magType['magType']=='mw']
group_mwb = df_mag_magType['mag'][df_mag_magType['magType']=='mwb']
group_mwc = df_mag_magType['mag'][df_mag_magType['magType']=='mwc']
group_mwr = df_mag_magType['mag'][df_mag_magType['magType']=='mwr']
group_mww = df_mag_magType['mag'][df_mag_magType['magType']=='mww']

# Levene
statistic, p_value = levene(group_m, group_mb, group_md, group_ml, group_ms, group_mw, group_mwb, group_mwc, group_mwr, group_mww)
print('statistic: ',statistic)
print('p_value: ', p_value)

df_mag_magType = data_earthquake[['mag', 'magType']]
model = sm.OLS.from_formula('Q("mag") ~ C(magType)', data = df_mag_magType).fit(cov_type='HC3')
anova_result = sm.stats.anova_lm(model, typ=2)
print(anova_result2)

statistic:  3.868718663680521
p_value:  7.242522159317592e-05
                sum_sq      df         F         PR(>F)
C(magType)  104.532206     9.0  79.02912  4.279435e-121
Residual    233.971805  1592.0       NaN            NaN


### CONCLUSIONES:
#### Se rechazara la hipotesis nula, esto se debe a que los tipos de magnitudes estan comprendidos usualmente dentro de una rango de medida por lo tanto existe una diferencia.